# Othello board representation

In [1]:
import numpy as np
import copy
import random
import math

class Othello:
    """Representation of an Othello board
    
    Attribute:
    _self.board: An array with the current state of the board
    Player 1 (Min) is noted as 1 and player 2 (Max) is ntoed as 2
    
    Methods:
    
    _visualize(self): draw out the current state of the board with 
    player 1 as 0 and player 2 as X
    _visualize_possible_move(self,player,possible_move): Same as visualize() above, 
    just add highlights (*) to the possible move of a given player.
    _is_valid(self,current_player,x,y): Given a player and a move coordinate, check
    if it's a valid move for the player. Return result + the list of tokens to capture
    _valid_move(self,player,verbose=False): return a list of all possible moves for a player
    _move(self,player,x,y): Make a move, update the board
    _count(self,player): Count the number of 0/X pieces on the board
    _score(self,verbose=False): Return a tuple of (player1_score,player2_score)
    _evaluate(self): A heuristic function for Minimax algorithm 
    player1_score - player2_score since player 1 is Max and 2 is Min.
    _get_result(self,player): Return current result of the game for this player
    1 if win, 0.5 if draw, 0 is lose. Used in Monte Carlo Tree Search simulation step.
    _terminate(self): Check if the game is over
    
    """
    def __init__(self):
        self.board = np.zeros((8,8))
        # 1 is white (0), 2 is black (X)
        self.board[3,3] = 1
        self.board[4,4] = 1
        self.board[3,4] = 2
        self.board[4,3] = 2
    
    def visualize(self):
        for i in range(len(self.board)):
            print('\n')
            for j in range(len(self.board)):
                if self.board[i,j] == 0:
                    print(' _ ',end='')
                if self.board[i,j] == 1:
                    print(' 0 ',end='')
                if self.board[i,j] == 2:
                    print(' X ',end='')

    def visualize_possible_move(self,player,possible_move):
        copy_board = copy.deepcopy(self.board)
        print('\n')
        print("Valid move for player: ",player)

        for i,j in possible_move:
            copy_board[i,j] = 3
        for i in range(len(copy_board)):
            print('\n')
            for j in range(len(copy_board)):
                if copy_board[i,j] == 0:
                    print(' _ ',end='')
                if copy_board[i,j] == 1:
                    print(' 0 ',end='')
                if copy_board[i,j] == 2:
                    print(' X ',end='')
                if copy_board[i,j] == 3:
                    print(' * ',end='')
        print('\n')

    def is_valid(self,current_player,x,y):
        # Check if the move at (x,y) is valid for the player
        tile_to_flip = []
        # Check the vertical direction, downward
        
        if x < 7:
            con = False
            temp = []
            for i in range(x+1,len(self.board)):
                if self.board[i,y] == 0:
                    break 
                if not(con) and self.board[i,y] == current_player:
                    break
                if con and self.board[i,y] == current_player:
                    for tile in temp:
                        tile_to_flip.append(tile)
                    break
                else:
                    con = True
                    temp.append((i,y))

        # Check the vertical direction, upward
        if x > 0:
            con = False
            temp = []
            for i in reversed(range(0,x)):
                if self.board[i,y] == 0:
                    break
                if not(con) and self.board[i,y] == current_player:
                    break
                if con and self.board[i,y] == current_player:
                    for tile in temp:
                        tile_to_flip.append(tile)
                    break
               
                else:
                    con = True
                    temp.append((i,y))
        # Check the horizontal direction, to the left of the move
        if y > 0:
            con = False
            temp = []
            for i in reversed(range(0,y)):
                if self.board[x,i] == 0:
                    break
                if not(con) and self.board[x,i] == current_player:
                    break
                if con and self.board[x,i] == current_player:
                    for tile in temp:
                        tile_to_flip.append(tile)
                    break
                else:
                    con = True
                    temp.append((x,i))
        # Check the horizontal direction, to the right of the move
        if y < 7:
            con = False
            temp = []
            for i in range(y+1,len(self.board)):
                if self.board[x,i] == 0:
                    break
                if not(con) and self.board[x,i] == current_player:
                    break
                if con and self.board[x,i] == current_player:
                    for tile in temp:
                        tile_to_flip.append(tile)
                    break
                else:
                    con = True
                    temp.append((x,i))

        # Check diagonally to upper left
        if x>0 and y>0:
            v = x-1
            h = y-1
            con = False
            temp = []
            while v>=0 and h>=0:
                if self.board[v,h] == 0:
                    break
                if not(con) and self.board[v,h] == current_player:
                    break
                if con and self.board[v,h] == current_player:
                    for tile in temp:
                        tile_to_flip.append(tile)
                    break
                else:
                    con = True
                    temp.append((v,h))
                    v -= 1
                    h -= 1
        
        # Check diagonally, upper right
        if x>0 and y<7:
            v = x-1
            h = y+1
            con = False
            temp = []
            while v>=0 and h<=7:
                if self.board[v,h] == 0:
                    break
                if not(con) and self.board[v,h] == current_player:
                    break
                if con and self.board[v,h] == current_player:
                    for tile in temp:
                        tile_to_flip.append(tile)
                    break
                else:
                    con = True
                    temp.append((v,h))
                    v -= 1
                    h += 1
        
        

        # Check diagonally, lower left
        if x<7 and y>0:
            v = x+1
            h = y-1
            con = False
            temp = []
            while v<=7 and h>=0:
                if self.board[v,h] == 0:
                    break
                if not(con) and self.board[v,h] == current_player:
                    break
                if con and self.board[v,h] == current_player:
                    for tile in temp:
                        tile_to_flip.append(tile)
                    break
                else:
                    con = True
                    temp.append((v,h))
                    v += 1
                    h -= 1
        # Check diagonally, lower right
        if x<7 and y<7:
            v = x+1
            h = y+1
            con = False
            temp = []
            while v<=7 and h<=7:
                if self.board[v,h] == 0:
                    break
                if not(con) and self.board[v,h] == current_player:
                    break
                if con and self.board[v,h] == current_player:
                    for tile in temp:
                        tile_to_flip.append(tile)
                    break
                else:
                    con = True
                    temp.append((v,h))
                    v += 1
                    h += 1
        if len(tile_to_flip) == 0:
            return (False,[])
        else:
            return (True,tile_to_flip)

    def valid_move(self,player,verbose=False):
        # Return a list of possible move for the player
        current_pos = np.where(self.board==player)
        blank_pos = np.where(self.board==0)
        possible_move = []
        for i,j in zip(blank_pos[0],blank_pos[1]):
            if self.is_valid(player,i,j)[0]:
                possible_move.append([i,j])
        if verbose:
            self.visualize_possible_move(player,possible_move)
        return possible_move

    def move(self,player,x,y):
        # Make a move for a given player in the x,y coordinate
        # Check if the move is valid:
        valid, tile_to_flip = self.is_valid(player,x,y)
        if valid:
            self.board[x,y] = player
            # Change the tile to that of the player
            for tile in tile_to_flip:
                x,y = tile
                self.board[x,y] = player
        else:
             raise ValueError("Invalid move")
    
    def count(self,player):
        # Count the number of white/black on the current board
        return len(np.where(self.board==player)[0])
    
    def score(self,verbose=False):
        player1_score = self.count(1)
        player2_score = self.count(2)
        
        if verbose:
            print('Player 1 score: ',player1score)
            print('Player 2 score: ',player2score)

        return (player1_score,player2_score)
    
    def evaluate(self):
        return self.count(1) - self.count(2)
    
    def get_result(self,player):
        opponent = 3 - player
        if (self.count(player) - self.count(opponent)) > 0:
            result = 1
        elif self.count(player) == self.count(opponent):
            result = 0.5
        else:
            result = 0
        return result
    
    def terminate(self):
        # Check if the game has terminated
        if len(self.valid_move(1)) == 0 and len(self.valid_move(2)) == 0:
            return True
        return False




# Example for board class

In [22]:
board = Othello()
board.valid_move(2,verbose=True)
board.move(2,4,5)
board.valid_move(1,verbose=True)
board.move(1,5,5)
board.valid_move(2,verbose=True)
print("Is the game over? ", board.terminate())
print("Player 1 score so far: ", board.score()[0])
print("Player 2 score so far: ", board.score()[1])



Valid move for player:  2


 _  _  _  _  _  _  _  _ 

 _  _  _  _  _  _  _  _ 

 _  _  _  *  _  _  _  _ 

 _  _  *  0  X  _  _  _ 

 _  _  _  X  0  *  _  _ 

 _  _  _  _  *  _  _  _ 

 _  _  _  _  _  _  _  _ 

 _  _  _  _  _  _  _  _ 



Valid move for player:  1


 _  _  _  _  _  _  _  _ 

 _  _  _  _  _  _  _  _ 

 _  _  _  _  _  _  _  _ 

 _  _  _  0  X  *  _  _ 

 _  _  _  X  X  X  _  _ 

 _  _  _  *  _  *  _  _ 

 _  _  _  _  _  _  _  _ 

 _  _  _  _  _  _  _  _ 



Valid move for player:  2


 _  _  _  _  _  _  _  _ 

 _  _  _  _  _  _  _  _ 

 _  _  _  *  _  _  _  _ 

 _  _  *  0  X  _  _  _ 

 _  _  _  X  0  X  _  _ 

 _  _  _  _  *  0  _  _ 

 _  _  _  _  _  *  _  _ 

 _  _  _  _  _  _  _  _ 

Is the game over?  False
Player 1 score so far:  3
Player 2 score so far:  3


# Minimax with alpha-beta pruning

**Pseudo-code for minimax with - pruning**

function alphabeta(node, depth, α, β, maximizingPlayer) is
> if depth = 0 or node is a terminal node then <br> 
> > return the heuristic value of node <br>

> if maximizingPlayer then <br>
> > value := −∞ <br>

> for each child of node do <br>
> > value := max(value, alphabeta(child, depth − 1, α, β, FALSE)) <br>
> > α := max(α, value) <br>
> > if α ≥ β then <br> 
> > > break (* β cut-off *) <br>
> > return value <br>

> else <br>
> > value := +∞ <br>
> for each child of node do <br>
> > value := min(value, alphabeta(child, depth − 1, α, β, TRUE)) <br>
> > β := min(β, value) <br>
> > if α ≥ β then <br>
> > > break (* α cut-off *) <br>

> return value <br>


In [2]:
def minimax_value(position,board,depth,alpha,beta,maxPlayer):
    """
    Inputs:
    
    _position: the move coordinates in form [row,col]
    _board: the current board state
    _depth: the desired depth of the search tree
    _alpha: parameter
    _beta: parameter
    _maxPlayer: True if the current player is Max (player 1)
    
    Outputs:
    _Evaluation of the node
    
    """
    if maxPlayer:
        player = 1
    else:
        player = 2
    
    # We have reached the depth of the search tree
    if depth == 0:
        # Evaluation of the board (our heuristic) = player1 score - player2 score
        # Player 1 is trying minimize evaluation whereas player 2 try to maximize
        return board.evaluate()
    # Check all valid moves of the current player
    child = board.valid_move(player)
    # If no move is possible from here, return the current evaluation of the board
    if len(child) == 0:
        return board.evaluate()
    if maxPlayer:
        maxEval = float('-inf')
        for c in child:
            copy_board = copy.deepcopy(board)
            copy_board.move(player,c[0],c[1])
            evaluation = minimax_value(c,copy_board,depth-1,alpha,beta,maxPlayer=False)
            maxEval = max(maxEval,evaluation)
            alpha = max(alpha,evaluation)
            # Prunning
            if beta <= alpha:
                break
        return maxEval
    else:
        minEval = float('inf')
        for c in child:
            copy_board = copy.deepcopy(board)
            copy_board.move(player,c[0],c[1])
            evaluation = minimax_value(c,copy_board,depth-1,alpha,beta,maxPlayer=True)
            minEval = min(minEval,evaluation)
            beta = min(beta,evaluation)
            if beta <= alpha:
                break
        return minEval



def minimax_decision(current_player,board,possible_move,depth):
    if current_player == 1:
        maxPlayer = True
    else:
        maxPlayer = False
    bestMoveVal = float('-inf')
    for move in possible_move:
        if move==[0,0] or move==[0,7] or move==[7,0] or move==[7,7]:
            return move
        else:
            copy_board = copy.deepcopy(board)
            copy_board.move(current_player,move[0],move[1])
            val = minimax_value(move,copy_board,depth,alpha=float('-inf'),beta=float('inf'),maxPlayer=maxPlayer)
        if val > bestMoveVal:
            bestMoveVal = val
            bestX = move[0]
            bestY = move[1]
    return [bestX,bestY]

# Monte Carlo Tree Search algorithm

**Algorithm description and pseudocode:**

> Create rootnode that contains current game state: node = rootnode <br>
> Repeat until resources exhausted: <br>

>> While the node is fully expanded (no untried move) and none-terminal (still have children nodes left): <br>
> > > Select the child with highest UCB1 upperbound (Selection) <br>
> > > node = selected child node <br>

>> If there is still untried moves:<br>
>>> Randomly make a move & add it as a new node (Expansion) <br>
>>> node = newly added node

>> Use Monte Carlo method to simulate a full game (Simulation) <br>
>> Assess the result of the game <br>
>> Backpropagate the result all the way back to the rootnode (Backpropagation) <br>

> Return the child of the root node that is most frequently visited <br>



<br>Data structure used: Tree. The root node of the tree stores the current state of the game. The children of this root node will store the move that led to it, the statistics of win times with that move / total visit times along with the state of the game after that move.
<br>
<br>
In the next section, we would take a deeper look into each operation in the algorithm, namely: Selection, Expansion, Simulation, Backpropagation. <br>

In [3]:
class Node(object):

    def __init__(self, move=None, parent=None, state=None, previous_player=None):
        self.move = move # the move that led to this node
        self.parent = parent
        self.children = [] # List of children, i.e the subsequent move consequence
        # Initialize statistics
        self.win = 0
        self.visit = 0
        self.previous_player = previous_player
        self.current_player = 3 - previous_player
        self.untried_moves = state.valid_move(self.current_player) # Get all possible moves for this state

    def uct_select_child(self):
        """
        Use upper confidence bound (UBC1) formula to select a child node.
        The formula for UCB is: x + sqrt(2*log(n)/n_i)
        with:
        x is the expected gain (ratio of winning in this case)
        n is the total number of visits
        n_i is the total number of visits for the child node i

        Return the child with the highest 'score'
        """
        select = sorted(self.children,key = lambda c: c.win/c.visit + math.sqrt(2*math.log(self.visit)/c.visit))[-1]
        return select

    def add_child(self, m, s):
        """
        Input:
        m: the move that lead us to this new node
        s: the current state of the game
        Remove m from list of untried_moves and add a new child node for this move
        Return this newly added node
        """
        n = Node(move=m, parent=self, state=s, previous_player=self.current_player)
        self.untried_moves.remove(m)
        self.children.append(n)
        return n

    def update(self,result):
        """
        Update the statistics (win, visit) of this node
        Result is viewed from the point of previous_player
        """
        self.visit += 1
        self.win += result

    def __repr__(self):
        """
        Print out the current node. Info include: move, statistics, untried_moves
        """
        s = "[Move: "+str(self.move)+" Win/Visit: "+str(self.win)+"/"+str(self.visit)+" untried_moves: "+str(self.untried_moves)+"]"
        return s

    def print_children(self):
        s = ""
        for child in self.children:
            s += str(child) + "\n"
        return s

def mcts(rootstate,itermax,previous_player,show=False):
    
    root = Node(state=rootstate,previous_player=previous_player)
    current_player = 3 - previous_player
    
    # Running search tree when we still have resources left
    for i in range(itermax):
        node = root
        state = copy.deepcopy(rootstate)
        # Descending, select the child with highest UCT score
        # Do this action when the node is fully expand and non-terminal
        while node.untried_moves == [] and node.children != []: # while we have tried all move for this node and now we will look at its children
            node = node.uct_select_child()
            state.move(node.previous_player,node.move[0],node.move[1])

        # Expand
        # Check if there is still untried moves -> we can expand
        if node.untried_moves != []:
            # Randomly make a move
            m = random.choice(node.untried_moves)
            state.move(node.current_player,m[0],m[1])
            current_player, previous_player = previous_player, current_player
            node = node.add_child(m,state)
            
        # Using Monte Carlo method to simulate a full game
        while state.valid_move(current_player) and not state.terminate():
            m = random.choice(state.valid_move(current_player))
            state.move(current_player,m[0],m[1])
            current_player, previous_player = previous_player, current_player
            
        # Backpropagate the result up the nodes
        while node != None:            
            node.update(state.get_result(node.previous_player))
            node = node.parent

    # Return the move for the node that is most frequently visited
    if show == True:
        print(root.print_children())
    best_move = sorted(root.children,key=lambda x: x.visit)[-1].move
    return best_move



In [4]:
def play(strategy,depth,verbose=False):
    """Simulate a single gameplay
    Inputs:
    _strategy: "Minimax" or "MCTS" to indicate the search strategy
    _depth: either the depth of the search tree (for minimax) or
            the max number of iterations for MCTS
    _verbose: boolean value, for debugging
    Outputs: winning rate against random strategy
    """
    b = Othello()
    current_player = 2
    other_player = 1

    while not(b.terminate()):
        possible_move = b.valid_move(current_player)
        if len(possible_move) == 0:
            current_player, other_player = other_player, current_player
            continue
        if current_player == 1 and strategy == "Minimax":
            x,y = minimax_decision(current_player,b,possible_move,depth)
        elif current_player == 1 and strategy == "MCTS":
            x,y = mcts(b,depth,other_player,verbose)
        else:
            index = np.random.choice(len(possible_move))
            x,y = possible_move[index]
        b.move(current_player,x,y)
        current_player, other_player = other_player, current_player
        if verbose:
            b.visualize()
            print('\n')
    return b.score()

from tqdm.notebook import tqdm
def simulate(n,strategy,depth,verbose=False):
    win = 0
    for i in tqdm(range(n)):
        player1_score, player2_score = play(strategy,depth)
        if player1_score > player2_score:
            win += 1
    if verbose:
        print("Win rate using %s: ".format(strategy),win,"/",n)
    return win

In [23]:
%%time
minimax_depth_3 = simulate(100,"Minimax",3)
print("Win rate using Minimax with depth 3 against random strategy: ",minimax_depth_3)


Win rate using Minimax with depth 3 against random strategy:  85
CPU times: user 5min 36s, sys: 8.19 s, total: 5min 44s
Wall time: 5min 38s


In [6]:
%%time
minimax_depth_5 = simulate(100,"Minimax",5)
print("Win rate using Minimax with depth 5 against random strategy: ", minimax_depth_5)


Win rate using Minimax with depth 5 against random strategy:  89
CPU times: user 1h 32min 31s, sys: 52.1 s, total: 1h 33min 23s
Wall time: 1h 34min 57s


In [7]:
%%time
MCTS_20 = simulate(100,"MCTS",20)
print("Win rate using Monte Carlo Tree Search with itermax = 20 against random strategy: ",MCTS_20)


Win rate using Monte Carlo Tree Search with itermax = 20 against random strategy:  88
CPU times: user 21min 57s, sys: 3.21 s, total: 22min
Wall time: 22min 5s


In [8]:
%%time
MCTS_50 = simulate(100,"MCTS",50)
print("Win rate using Monte Carlo Tree Search with itermax = 50 against random strategy: ",MCTS_50)


Win rate using Monte Carlo Tree Search with itermax = 50 against random strategy:  95
CPU times: user 52min 29s, sys: 4.36 s, total: 52min 33s
Wall time: 52min 39s


In [9]:
%%time
MCTS_100 = simulate(100,"MCTS",100)
print("Win rate using Monte Carlo Tree Search with itermax = 100 against random strategy: ",MCTS_100)


Win rate using Monte Carlo Tree Search with itermax = 100 against random strategy:  98
CPU times: user 1h 40min 45s, sys: 2.11 s, total: 1h 40min 47s
Wall time: 1h 40min 49s


In [10]:
%%time
def mcts_vs_minimax(itermax,depth,verbose=False):
    b = Othello()
    current_player = 2
    other_player = 1
    
    while not(b.terminate()):
        possible_move = b.valid_move(current_player)
        if len(possible_move) == 0:
            current_player, other_player = other_player, current_player
            continue
        if current_player == 1:
            x,y = mcts(b,itermax,other_player,verbose)
        else:
            x,y = minimax_decision(current_player,b,possible_move,depth)
        b.move(current_player,x,y)
        current_player, other_player = other_player, current_player
        if verbose:
            b.visualize()
            print('\n')
    return b.score()

def simulate_2_strategies(n,itermax,depth,verbose=False):
    # player 1 is MCTS, player 2 is Minimax
    win = 0
    for i in tqdm(range(n)):
        player1_score, player2_score = mcts_vs_minimax(itermax,depth)
        if player1_score > player2_score:
            win += 1
    if verbose:
        print("Win rate for MCTS vs Minimax: ",win,"/",n)
    return win

MCTS_vs_Minimax = simulate_2_strategies(100,100,5)
print("Win rate of MCTS against Minimax is: ",MCTS_vs_Minimax)


Win rate of MCTS against Minimax is:  96
CPU times: user 4h 1min 52s, sys: 1min 59s, total: 4h 3min 51s
Wall time: 4h 2min 15s
